<a href="https://colab.research.google.com/github/ganeshraju/Aadhar-uidaiBenchmark/blob/master/LLM_and_Healthcare_NLP_tasks_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook demonstrates toy examples of LLM Use cases in Healthcare using  OpenAI API. This is an experiment/prototype of ideas not a formal evaluation of the APIs or production code**

# Install Libraries

In [ ]:
!pip install openai
!pip install python-dotenv
!pip install --upgrade langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.6 MB/s eta 0:00:00


#Setup

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
#Autneticate notebook environment. Required for Google Cloud
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [ ]:
import pandas as pd
import openai
import os
import sys
import re
from langchain.document_loaders import GoogleDriveLoader
from langchain.document_loaders import GCSDirectoryLoader
from langchain.document_loaders import GCSFileLoader
from langchain.document_loaders import BigQueryLoader

# You may not need all the data sources . Choose whatever data source you want to use

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Store your API key in Google Drive file: openaiapi.txt . This is a secure way to use an API key that explicit API key in the notebook

with open('/content/drive/My Drive/openaiapi.txt', 'r') as file:
    OPENAI_API_KEY = file.read().strip()

In [ ]:
openai.api_key = OPENAI_API_KEY
completions_model = "gpt-4"
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"

In [ ]:
# Helper function to call the model API
# For Healthcare NLP tasks you may run into token limit as clinical notes are long.

def get_completion(messages, model, temperature):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature= temperature,
    )
    content = response.choices[0].message["content"]
    return content

# Use Case #1 - NLP entity and context extractions

# Chain of Thought Reasoning to provide additional context in the NLP extractions

In [ ]:
text = f"""
 The patient is a 17-year-old female, who presents to the emergency room with foreign body and airway compromise and was taken to the operating room.  She was intubated and fishbone.
 """

In [ ]:
system_message = f"""
You are a Healthcare AI Assistant helping to extract entities and context from clinical text using the guidance stated below:
1 - Entity Recognition: You will have entity categories: Person, Location, Age,Gender, Disease/Problem, Anatomical Structure,Symptoms,Procedure, Medications,\
Medical Devices, Lab Test, Substance Abuse, Social Determinants.

2 - Entity Assertions: Probability of Assertions in extracted entities.Classify the assertions made on given medical concepts as being present, absent,
or possible in the patient, conditionally present in the patient under certain circumstances, hypothetically present in the patient at some future point,\
and mentioned in the patient report but associated with someone other than the patient.In addition, perform Subject Asssessment. Differentiate between \
"Patient" Vs. " Family Member" in the text description.
For example: "John's father has diabetes". Attach "diabetes" to assertion status: Family Member'
Assertion Status: 1. Present, 2. Absent, 3. Possible, 4. Hypothetical, 5. Conditional, 6. Family

3 - Temporal Assessment: Extract Date or Temporality of the entity and use these categorization:
    Extract Actual Date if date is available in the text.
In case where date is not available, assess temporality and categorize as:
1. Current
2. History
3. Actual Date if date is available in the text

Input Data: A 60 year old male with a history of type-2 diabetes, diagnosed 10 years ago, takes 500 mg metformmin.
Output:
{
    {
      "Name": "60-year-old",
      "Category": "Demographic Entity",
      "Assertion Status": "Present",
      "Temporality": "Current"
    },
    {
      "Name": "male",
      "Category": "Demographic Entity",
      "Assertion Status": "Present",
      "Temporality": "Current"
    },
    {
      "Name": "type-2 diabetes",
      "Category": "Disease/Problem",
      "Assertion Status": "Present",
      "Temporality": "10 years ago"
    },
    {
      "Name": "metformin",
      "Category": "medication",
      "Assertion Status": "Present",
      "Temporality": "Current"
    },
}
"""

# Prompt

In [ ]:
# Initialize a prompt.
prompt = f"""
Perform the following actions. Lets think step by step and use the guidance provided. Take your time and try to answer accurately.
1 -  Extract entities, classify, probability of assertions, assess temporality, and assess subject of the entity.
2 - Output the json object that contains the following keys: Entity, Category, Subject, Temporality, Assertion Probability, Related Entities (multiple values)
If the question cannot be answered using the information provided answer with “No entities found”
text: {text}
"""
messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content':prompt},
]

# System Response: NLP Extraction

In [ ]:
temperature = 0.  # To make the output more deterministic
assistant_response = get_completion(messages, completions_model, temperature)
print(assistant_response)

1 - Extract entities, classify, assess negation probability, assess temporality, and assess subject of the entity:

- Entity: 17-year-old female
  Category: Demographic Entity
  Subject: Patient
  Temporality: Current
  Negation Probability: N/A

- Entity: foreign body
  Category: Anatomical Structure
  Subject: Patient
  Temporality: Current
  Negation Probability: N/A

- Entity: airway compromise
  Category: Disease/Problem
  Subject: Patient
  Temporality: Current
  Negation Probability: N/A

- Entity: operating room
  Category: Procedures
  Subject: Patient
  Temporality: Current
  Negation Probability: N/A

- Entity: intubated
  Category: Procedures
  Subject: Patient
  Temporality: Current
  Negation Probability: N/A

- Entity: fishbone
  Category: Anatomical Structure
  Subject: Patient
  Temporality: Current
  Negation Probability: N/A

2 - Output the JSON object:

{
  "Entity": [
    {
      "Name": "17-year-old female",
      "Category": "Demographic Entity",
      "Subject":

# Use Case #2: Write SOAP Notes from Patient-Clinician Conversation

In [ ]:
# Set the context

system_message = f"""
You are a Healthcare AI Assistant helping to write SOAP Notes from the conversation text.
"""

In [ ]:
# Input Data

conversation_1 = f"""
Hi, how's it going?    I'm not feeling well today. I have some abdominal pain.     I'm sorry to hear that.  Can you tell me a little bit about that? Yes, I have had some pain for the last two weeks, in the mid abdomen, going to the lower abdomen. Have you had any nausea or vomiting or diarrhea? Yes, I've had some diarrhea. Anybody else at home that sick? Well, my husband and my son are also sick with some diarrhea and abdominal pain. Do you have any fevers? No, I don't have any fevers or chills. OK let's take a look and examine you.
Well, I think you might have some gastroenteritis,  And infection of the abdomen just caused by food poisoning. I think if you drink plenty of water, and stick to a brat diet, it should pass. However, if it still lingers after several days, I think we need to run some tests. How does that sound? Thank you doctor, that sounds like a plan .
"""

In [ ]:
prompt = f"""
Perform the following actions. Lets think step by step and use the guidance provided. Take your time and try to answer accurately.
1 - Create SOAP Note from Patient-docto conversation
text: {conversation_1}
"""
messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content':prompt},
]

In [ ]:
temperature = 0.2  # Allow for little creativity
assistant_response = get_completion(messages, completions_model, temperature)
print(assistant_response)

Subjective:
- Chief complaint: Abdominal pain and diarrhea for the last two weeks
- Location: Mid abdomen, going to the lower abdomen
- Associated symptoms: Diarrhea
- Family members affected: Husband and son also experiencing diarrhea and abdominal pain
- No fevers or chills

Objective:
- Physical examination: Abdominal examination performed

Assessment:
- Possible gastroenteritis, likely due to food poisoning

Plan:
- Drink plenty of water
- Follow a BRAT diet (bananas, rice, applesauce, toast)
- If symptoms persist after several days, consider running tests


# The reason for this SOAP notes

# Use Case #2A: Write a after-visit summary for the patient

In [ ]:
# Set the context

system_message = f"""
You are a Healthcare AI Assistant helping to write patient communication.
"""

In [ ]:
prompt = f"""
Perform the following actions. Lets think step by step and use the guidance provided. Take your time and try to answer accurately.
1 - Write a after-visit summary and follow up email for the patient. Write it in a format so that the summary could be emailed to patient. Details on escalation.
text: {conversation_1}
"""
messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content':prompt},
]

In [ ]:
temperature = 0.4  # Allow for little creativity
assistant_response = get_completion(messages, completions_model, temperature)
print(assistant_response)

Subject: After-Visit Summary - [Patient Name]

Dear [Patient Name],

I hope this email finds you well. I wanted to provide you with a summary of your recent visit to our clinic. During your visit, you mentioned experiencing abdominal pain for the last two weeks, along with diarrhea. Your husband and son have also been experiencing similar symptoms.

After examining you, I believe that you might have gastroenteritis, which is an infection of the abdomen likely caused by food poisoning. As we discussed, I recommend the following steps to help alleviate your symptoms:

1. Drink plenty of water to stay hydrated.
2. Follow a BRAT diet (Bananas, Rice, Applesauce, and Toast) to help manage your diarrhea and abdominal pain.

If your symptoms persist or worsen after several days, please contact our clinic to schedule a follow-up appointment. We may need to run additional tests to determine the cause of your symptoms.

Please don't hesitate to reach out if you have any questions or concerns. We 

# Use Case # 2B Improving Clinician's Performance

In [ ]:
# Set the context

system_message = f"""
You are a Healthcare AI Assistant helping to improve clincian's performance.
"""

In [ ]:
prompt = f"""
Perform the following actions. Lets think step by step and use the guidance provided. Take your time and try to answer accurately.
1 - Write an assessment of clincian's performance in this encounter and provide suggestions on how to improve for the future?
text: {conversation_1}
"""
messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content':prompt},
]

In [ ]:
temperature = 0.4  # Allow for little creativity
assistant_response = get_completion(messages, completions_model, temperature)
print(assistant_response)

Assessment of Clinician's Performance:

Strengths:
1. The clinician started the conversation in a friendly manner, asking the patient how they are doing.
2. The clinician asked relevant questions to gather more information about the patient's symptoms, such as the duration and location of the pain, presence of nausea, vomiting, diarrhea, and fever, and if other family members are experiencing similar symptoms.
3. The clinician performed a physical examination to further assess the patient's condition.
4. The clinician provided a possible diagnosis and suggested a course of action, including hydration, dietary changes, and follow-up if symptoms persist.

Areas for Improvement:
1. The clinician could have asked more probing questions to better understand the severity and frequency of the symptoms, such as the number of diarrheal episodes per day, the presence of blood or mucus in the stool, and any recent changes in diet or travel history.
2. The clinician could have shown more empathy a

# Use Case 2C - Helping with any prior authorization

In [ ]:
# Set the context

system_message = f"""
You are a Healthcare AI Assistant helping with prior authorization.
"""

In [ ]:
prompt = f"""
Perform the following actions. Lets think step by step and use the guidance provided. Take your time and try to answer accurately.
1 - Assess if prior authoirization is required for anything clinician ordered. If we need a prior authorization, write a brief justification.
text: {conversation_1}
"""
messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content':prompt},
]

In [ ]:
temperature = 0.4  # Allow for little creativity
assistant_response = get_completion(messages, completions_model, temperature)
print(assistant_response)

Based on the provided text, the clinician has not ordered any medications, procedures, or tests that would require prior authorization at this time. The current recommendation is for the patient to drink plenty of water and follow a brat diet. If the patient's condition does not improve after several days, the clinician may consider ordering tests, which may or may not require prior authorization depending on the specific tests ordered.


# Use Case 2D - create HL7 FHIR Resources in JSON format, from the conversation summary

In [ ]:
# Set the context

system_message = f"""
You are a Healthcare AI Assistant helping with Data Processing.
"""

In [ ]:
prompt = f"""
Perform the following actions. Lets think step by step and use the guidance provided. Take your time and try to answer accurately.
1 - Create appropriate HL7 FHIR resources for the clincian order/tests.
text: {conversation_1}
"""
messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content':prompt},
]

In [ ]:
temperature = 0.1
assistant_response = get_completion(messages, completions_model, temperature)
print(assistant_response)

To create appropriate HL7 FHIR resources for the clinician order/tests, we will first identify the relevant information from the text and then create the necessary resources.

Relevant information:
- Patient: Female with abdominal pain, diarrhea, no fevers or chills
- Family members: Husband and son also sick with diarrhea and abdominal pain
- Possible diagnosis: Gastroenteritis
- Suggested treatment: Drink plenty of water, stick to a brat diet
- Potential tests: If symptoms persist after several days

HL7 FHIR resources:

1. Patient resource:
{
  "resourceType": "Patient",
  "id": "example",
  "text": {
    "status": "generated",
    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">Female patient with abdominal pain and diarrhea</div>"
  },
  "gender": "female"
}

2. Condition resource:
{
  "resourceType": "Condition",
  "id": "example",
  "text": {
    "status": "generated",
    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">Possible gastroenteritis</div>"
  },
  "subject": {

# Use Case # 3: Biomedical Research Assistant for Clincians

Create a Research Assistant Bot

In [ ]:
# Helper function to call chat model

def get_completion_from_messages(messages,
                                 model="gpt-4",
                                 temperature=0,
                                 max_tokens=1024):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut
    )
    return response.choices[0].message["content"]

In [ ]:
abstract = """

Title: Sepsis-Associated Acute Kidney Injury
link: https://pubmed.ncbi.nlm.nih.gov/33752856/

abstract: Sepsis-associated acute kidney injury (S-AKI) is a frequent complication of the critically ill patient and is associated with unacceptable morbidity and mortality.\
Prevention of S-AKI is difficult because by the time patients seek medical attention, most have already developed acute kidney injury. Thus, early recognition is crucial \
to provide supportive treatment and limit further insults. Current diagnostic criteria for acute kidney injury has limited early detection; however, novel biomarkers \
of kidney stress and damage have been recently validated for risk prediction and early diagnosis of acute kidney injury in the setting of sepsis. Recent evidence shows \
that microvascular dysfunction, inflammation, and metabolic reprogramming are 3 fundamental mechanisms that may play a role in the development of S-AKI. \
However, more mechanistic studies are needed to better understand the convoluted pathophysiology of S-AKI and to translate these findings into potential treatment strategies \
and add to the promising pharmacologic approaches being developed and tested in clinical trials.
"""

In [ ]:
# Set the context
system_message = f"""
You are an Healthcare Research AI Assistant. Summarize and explain research papers.
"""

user_message_1 = f"""
summarize the paper. explain why this paper is important. suggest other relevant paper to read. suggest a potential research investigation.
"""

In [ ]:
messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content': user_message_1},
{'role':'assistant',
 'content': f"Relevant research information: {abstract}" },
]
final_response = get_completion_from_messages(messages)
print(final_response)

Summary:
The paper titled "Sepsis-Associated Acute Kidney Injury" discusses the high prevalence of S-AKI in critically ill patients and its association with significant morbidity and mortality. The authors emphasize the importance of early recognition and supportive treatment to prevent further complications. They also highlight the limitations of current diagnostic criteria for AKI and the potential of novel biomarkers for early detection and risk prediction. The paper explores the role of microvascular dysfunction, inflammation, and metabolic reprogramming in the development of S-AKI and calls for more mechanistic studies to better understand its pathophysiology and develop effective treatment strategies.

Importance:
This paper is important because it addresses a critical complication in sepsis patients, which has a significant impact on morbidity and mortality. It highlights the need for early detection and intervention, as well as the potential of novel biomarkers to improve diagn

In [ ]:
prompt = f"""
Your task is to generate a short summary of a medical article \
abstract from pubmed site.
1 - Summarize the abstract below,in at most 20 words focusing on AKI prediction and treatment options.
2 - List the major topics discussed in the article. Classify the topics into:
    a. disease, b. symptom, c. procedure, d.medication.
article:{abstract}
"""
messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content':prompt},
]

In [ ]:
temperature = 0.2  # Allow for little creativity
assistant_response = get_completion(messages, completions_model, temperature)
print(assistant_response)

1 - Early recognition of S-AKI is crucial; novel biomarkers aid in risk prediction and early diagnosis, with potential treatment strategies being explored.

2 - S-AKI complications, prevention difficulties, early recognition, current diagnostic criteria, novel biomarkers, microvascular dysfunction, inflammation, metabolic reprogramming, pathophysiology, potential treatment strategies, pharmacologic approaches, clinical trials.


# Use Case # 4: Summarize a Patient's Medical Record

In [ ]:
clinical_text = f"""
Plan:
CKD (Serology): due to likely hyperfiltration syndrome (patient drinking 2 gallons of water),- improved creatinine
-I warned the patient about hyperfiltration syndrome.  The patient is drinking way too much liquid, and may be causing worsening of his renal function, because of hyperfiltration syndrome.
- he needs to limit his fluid intake to 2L, a bit more if he exercises and sweats
- his blood pressure and blood sugar are decently controlled now

Proteinuria:   well controlled
- protein restriction discussed
- on ACE inhibitor or angiotensin receptor blocker: yes

Blood Pressure:  well controlled
- low salt diet
- current treatment plan is effective, no change in therapy Diabetes/[Synop]: stable Continue current plan. Review blood glucose monitoring. Discuss risks of poor blood glucose control. Review carbohydrate-controlled diet.

Preventative: HMM  Care Gap SS  Check labs now Follow up with me in 3 months
Cc to Medical assistant
=============================================================  SUBJECTIVE

John Smith is a male with diabetes mellitus 2, hypertension, bph, hyperlipidemia, aaa, high bmi, CAD/CABG, AF on coumadin, copd, anemia, osa, mdd, gerd, moderate chronic renal failure for month(s) who comes to see me for a follow up visit.  Patient says that he drinks about 2 gallons a day. Because his mouth is really dry.

Lab Results  Component
Value
Date  Creatinine
2.25 (H)
04/11/2023  Creatinine
2.08 (H)
04/02/2023  Creatinine
2.34 (H)
04/01/2023   ROS: + frequency - urgency + dysuria - hematuria - skin changes/rash + joint pains; takes tylenol  - sinus problems - epistaxis - cough with blood + stone history; many years ago x 4, last 1969 + urinary hesitancy + nocturia: 2 times a nigth  + leg edema - little in the left leg  - NSAID use    Reviewed: medical history with no changes 5/12/2023, social history with no changes  5/12/2023 and family history with no changes 5/12/2023      PHYSICAL EXAM
     BP Readings from Last 3 Encounters:
05/12/23  103/55
04/07/23  101/54
04/02/23  132/72
     Pulse Readings from Last 3 Encounters:
05/12/23  80
04/07/23  69
04/02/23  72
     Wt Readings from Last 3 Encounters:
05/12/23  116.9 kg (257 lb 11.2 oz)
04/07/23  115.7 kg (255 lb)
04/02/23  123.8 kg (273 lb)
     BMI Readings from Last 3 Encounters:
05/12/23  39.18 kg/m²
04/07/23  38.77 kg/m²
04/02/23  41.51 kg/m²
  General appearance - oriented to person, place, and time Chest - clear Heart - S1 and S2 normal Abdomen - soft Extremities - pedal edema: 0 +     RESULTS
Data Reviewed:  Reviewed lab results: Renal:       Lab Results
Component  Value  Date
   Estimated Glomerular Filtration Rate  29 (L)  04/11/2023
   Estimated Glomerular Filtration Rate  24 (L)  03/31/2023
   Estimated Glomerular Filtration Rate  35 (L)  11/23/2022
         Lab Results
Component  Value  Date
   Creatinine  2.25 (H)  04/11/2023
   Creatinine  2.08 (H)  04/02/2023
   Creatinine  2.34 (H)  04/01/2023
   Creatinine  2.63 (H)  03/31/2023
   Creatinine  1.94 (H)  11/23/2022
         Lab Results
Component  Value  Date
   BUN  29 (H)  04/02/2023
      Anemia:       Lab Results
Component  Value  Date
   Hgb  11.5 (L)  04/02/2023
   Hgb  11.5 (L)  04/01/2023
   Hematocrit  37.1 (L)  04/02/2023
   Hematocrit  36.3 (L)  04/01/2023
   MCV  90  04/02/2023
   MCV  89  04/01/2023
   Transferrin % saturation  37  11/23/2022
   Transferrin % saturation  7 (L)  11/22/2016
    Bone:       Lab Results
Component  Value  Date
   Calcium  8.7 (L)  03/31/2023
         Lab Results
Component  Value  Date
   Phosphorus  3.5  03/31/2023
   No results found for: PTHINTACT No results found for: VITD25    Potassium:       Lab Results
Component  Value  Date
   Potassium  5.0  04/11/2023
   Potassium  4.5  04/02/2023
   Potassium  4.2  04/01/2023
    Protein:      Lab Results
"""

In [ ]:
# Set the context
system_message = f"""
You are an Healthcare AI Assistant helping to summarize patient's situation from medical history.
"""

In [ ]:
prompt = f"""
Your task is to generate a short summary of patient's medical record \
medical record:{clinical_text}
"""
messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content':prompt},
]

In [ ]:
temperature = 0.2  # Allow for little creativity
assistant_response = get_completion(messages, completions_model, temperature)
print(assistant_response)

Summary: John Smith is a male patient with a history of diabetes mellitus type 2, hypertension, BPH, hyperlipidemia, AAA, high BMI, CAD/CABG, AF on Coumadin, COPD, anemia, OSA, MDD, GERD, and moderate chronic renal failure. He reports drinking 2 gallons of water daily due to dry mouth. Recent lab results show improved creatinine levels, well-controlled proteinuria, and stable blood pressure and blood sugar. The patient has been advised to limit fluid intake to 2L per day to prevent hyperfiltration syndrome. He is on an ACE inhibitor or angiotensin receptor blocker, and his current treatment plan is effective. The patient will follow up in 3 months.


# Use Case # 5: Semantic Search and Question and Answering, TBD

# Use Case # 6: Patient Navigation/Co-pilot Chatbot TBD

# Use Case # 7: Identify Patients meeting selection criteria of clinical trials

# Use Case # 8 : Semantic Similarity between text snippets to minimize data redundancy and enable summarization

# Evaluate the LLM's answer based on "expert" human generated answer. Work in Progress.

In [ ]:
# This is a evaluation framework, I still have to work with MIMIC data to create the test.
# Source https://github.com/openai/evals/blob/main/evals/registry/modelgraded/fact.yaml

def eval_with_ideal(test_set, assistant_answer):

    test_input = test_set['note']
    test_output = test_set['entity_list']
    llm_answer = assistant_answer

    system_message = """\
    You are an assistant that evaluates how well the data processing assistant \
    extracts entities by looking at the context that the customer service \
    agent is using to generate its response.
    """

    user_message = f"""\
You are evaluating a submitted answer to a question based on the context \
that the agent uses to answer the question.
Here is the data:
    [BEGIN DATA]
    ************
    [Input]: {test_input}
    ************
    [Expert Output]: {test_output}
    ************
    [Submission]: {llm_answer}
    ************
    [END DATA]

Compare the factual content of the submitted answer with the expert answer. Ignore any differences in style, grammar, or punctuation.
    The submitted answer may either be a subset or superset of the expert answer, or it may conflict with it. Determine which case applies. Answer the question by selecting one of the following options:
    (A) The submitted answer is a subset of the expert answer and is fully consistent with it.
    (B) The submitted answer is a superset of the expert answer and is fully consistent with it.
    (C) The submitted answer contains all the same details as the expert answer.
    (D) There is a disagreement between the submitted answer and the expert answer.
    (E) The answers differ, but these differences don't matter from the perspective of factuality.
  choice_strings: ABCDE
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion(messages, completions_model, temperature=0, max_tokens=4096)
    return response